# Model

## Import and read Data

In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
import os
from train_tabular import fit

os.environ["WANDB_SILENT"] = "true"

X_train = pd.read_csv(r'../../data/train_data.csv')
Y_train = X_train["Category"]

X_test = pd.read_csv(r'../../data/test_data.csv')
Y_test = X_test["Category"]
X_train= X_train.iloc[:,[i for i in range(5,14)]+ [-1]]
X_test= X_test.iloc[:,[i for i in range(5,14)]]

## Preprocessing

In [ ]:
# define preprocessing function
le = LabelEncoder()
Y_train = le.fit_transform(Y_train)
Y_test = pd.DataFrame(le.fit_transform(Y_test))

X_train['Patient_gender']= X_train['Patient_gender'].astype('category')
X_test['Patient_gender']= X_test['Patient_gender'].astype('category')

## Setup Model

In [ ]:
model = XGBClassifier(enable_categorical=True,random_state=42)

## Fit Model and Evaluate Model

In [ ]:
models = []
for fold in range(0,5):
    fitted_model_fold = fit(model,X_train,Y_train,X_test,Y_test,fold,"Tab-Data-XGBoost","XGBoost",None,class_names = le.fit_transform(['non-prolaktinom','prolaktinom']))
    models.append(fitted_model_fold)